In [488]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from numpy import quantile, where, random
import matplotlib.pyplot as plt
import random
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

data = pd.read_csv('data/result.csv')
data['outdoor_temp'] = data['outdoor_temp'] +273
data['zone_temp'] = data['zone_temp'] +273
data['supply_temp'] = data['supply_temp'] +273


train = data[84:108]
train = train.reset_index(drop=True)
test = data[53080:53092]
test = test.reset_index(drop=True)

# Defining Essential Functions

In [489]:
from numpy import random

##Faults:
# 1. Offset Fault: x' = a + x + err
# 2. Gain Fault: x' = Bx+ err
# 3. Stuck at Fault: x' = a
# 4. Out-of-bound: x'<theta1 and x'>theta2

#Function to simulate fault
def simulateFault(sensedValues,params):
    end_fault_index = int(len(sensedValues) * 0.5)
    interval = end_fault_index/4
    BMS = []
    classification = []
    
    noise = random.rand(len(sensedValues)) * params['err']
    for i in range(0, len(sensedValues)):
        if i <= interval:
            a = params['constant']
            sign = random.randint(1,3)
            if sign == 1:
                BMS.append(noise[i] + a + sensedValues[i])
            else:
                BMS.append(-noise[i] + a + sensedValues[i])
            classification.append(1)

        elif i > interval and i <= interval*2:
            B = params['coeff']
            sign = random.randint(1,3)
            if sign == 1:
                BMS.append(noise[i] + B * sensedValues[i])
            else:
                BMS.append(-noise[i] + B * sensedValues[i])
            classification.append(1)

        elif i > interval*2 and i<= interval*3:
            a = params['stuck']
            BMS.append(a)
            classification.append(1)
            
        elif i > interval*3 and i<= interval*4:
            type_fault = random.randint(1,3)
            low_bound = params['bounds'][0]
            up_bound = params['bounds'][1]
            if type_fault == 1:
                BMS.append(low_bound-noise[i])
            else:
                BMS.append(up_bound + noise[i])
            classification.append(1)
        else:
            BMS.append(noise[i] + sensedValues[i])
            classification.append(0)

    BMS = np.array(BMS)
    BMS = BMS.round(decimals = 3)
    sensedValues = sensedValues.round(decimals=3)
    return np.array(BMS), np.array(classification)

#Function to generate noise
def generateNoise(noise, values):
    noisy_data = []
    for i in range(0,len(values)):
        noisy_data.append(values[i]+random.uniform(-noise,noise))
    return noisy_data

#Function to model equation given the coeffs
def modeled_equation(vals,coeffs):
    x1 = []
    x2 = []
    x3 = []
    x4 = []
    x = []
    y = []
    for i in range(1,len(vals)):
        x1.append(-1231*(vals['zone_temp'][i] - vals['zone_temp'][i-1])/5 *coeffs[1])
        x2.append(72.57* vals['occupancy'][i]*coeffs[2])
        x3.append((vals['outdoor_temp'][i]-vals['zone_temp'][i])*coeffs[3])
        #x4.append(coeffs[0])
        y.append(-0.57*(vals['supply_temp'][i]-vals['zone_temp'][i]) * 1.005 * vals['air_flow'][i])
        x.append(x1[i-1]+x2[i-1]+x3[i-1])
    inputs = {'x1': x1,'x2':x2,'x3':x3}
    output = {'y':y}
    return inputs, output

#Function to define vectors for regression model
def define_vectors(vals):
    x1 = []
    x2 = []
    x3 = []
    y = []
    for i in range(1,len(vals)):
        x1.append(-1231*(vals['zone_temp'][i] - vals['zone_temp'][i-1])/5)
        x2.append(72.57*vals['occupancy'][i])
        x3.append((vals['outdoor_temp'][i]-vals['zone_temp'][i]))
        y.append(-0.57*(vals['supply_temp'][i]-vals['zone_temp'][i]) * 1.005 * vals['air_flow'][i])
    inputs = {'x1': x1,'x2':x2,'x3':x3}
    output = {'y':y}
    return inputs, output


In [490]:
inputs, output = define_vectors(train)
inputs_test, output_test = define_vectors(test)

# Scaling data 

In [494]:
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

scaler1 = StandardScaler()
scaler2 = StandardScaler()

inputs = pd.DataFrame(data = inputs)
inputs = sm.add_constant(inputs) #We add constant because statsmodel library requires us. Otherwise, it would take the form: y = mx.
output = pd.DataFrame(data = output)

inputs_test = pd.DataFrame(data = inputs_test)
inputs_test = sm.add_constant(inputs_test)
output_test = pd.DataFrame(data = output_test)

inputs_scaled = scaler1.fit_transform(inputs)
output_scaled = scaler2.fit_transform(output)

inputs_scaled_test = scaler1.transform(inputs_test)
output_scaled_test = scaler2.transform(output_test)

# Fitting training data to OLS regression

In [495]:
#x1: M_n
#x2: Q_n
#x3: R_n
model = sm.OLS(output_scaled, inputs_scaled)
results = model.fit()
results.summary()
#print(results.params

/Users/devanshukumar/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.997
Model:                            OLS   Adj. R-squared (uncentered):              0.997
Method:                 Least Squares   F-statistic:                              2329.
Date:                Thu, 06 May 2021   Prob (F-statistic):                    1.32e-25
Time:                        23:24:18   Log-Likelihood:                          34.744
No. Observations:                  23   AIC:                                     -63.49
Df Residuals:                      20   BIC:                                     -60.08
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const               0          0        nan        nan           0           0
x1             0.0551      0.053      1.049      0.307      -0.055       0.165
x2             0.9633      0.027     35.083      0.000       0.906       1.021
x3             0.0888      0.068      1.309      0.205      -0.053       0.230
==============================================================================
Omnibus:                        1.954   Durbin-Watson:                   0.635
Prob(Omnibus):                  0.377   Jarque-Bera (JB):                0.831
Skew:                          -0.429   Prob(JB):                        0.660
Kurtosis:                       3.362   Cond. No.                          inf
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

# Use Validation Data to check the MSE for the model

In [496]:
from sklearn.metrics import mean_squared_error as mse
pred = results.predict(inputs_scaled_test)
err = mse(output_scaled_test,pred)
print(err)

1.5907429242082498


# Two model comparision: Ground Truth Vs Fault Data

In this case, we take a random day (July 4th) and create two vectors: Ground Truth and Fault data. We assume that the sensor values already incorporate for noise and we just introduce fault for the fault_data vector. We use the build regression model to predict with the learnt coeffs and we use MSE metric to check the difference. 

In [500]:
ground_truth = data[120:140]
ground_truth = ground_truth.reset_index(drop=True)
#ground_truth['zone_temp'] = generateNoise(0.5, ground_truth['zone_temp'])

fault_data = data[120:140]
fault_data = fault_data.reset_index(drop=True)
stuck_val = random.randint(16,32)
# fault_data['zone_temp'],classification = simulateFault(fault_data['zone_temp'],{'constant': 4.5, 'coeff': 1.4, 'stuck': stuck_val, 'bounds': [16,40],'err':0.5})
fault_data['supply_temp'],classification = simulateFault(fault_data['supply_temp'],{'constant': 1, 'coeff': 1.01, 'stuck': 293.5, 'bounds': [293,294],'err':0.5})
# stuck_val = random.uniform(0,1)
# fault_data['air_flow'],classification = simulateFault(fault_data['air_flow'], {'constant': 0.09, 'coeff': 1.005, 'stuck': stuck_val, 'bounds': [0.05,0.8],'err':0.05})
# print(ground_truth)
# print(fault_data)

In [501]:
ground_truth_input, ground_truth_output = modeled_equation(ground_truth, results.params)
fault_input, fault_output = modeled_equation(fault_data, results.params)

ground_truth_input = pd.DataFrame(data = ground_truth_input)
ground_truth_input = sm.add_constant(ground_truth_input)
ground_truth_output = pd.DataFrame(data = ground_truth_output)

fault_input = pd.DataFrame(data = fault_input)
fault_input = sm.add_constant(fault_input)
fault_output = pd.DataFrame(data = fault_output)

ground_truth_input = scaler1.transform(ground_truth_input)
ground_truth_output = scaler2.transform(ground_truth_output)
fault_input = scaler1.transform(fault_input)
fault_output = scaler2.transform(fault_output)

In [502]:
pred_ground = results.predict(ground_truth_input)
print(mse(ground_truth_output,pred_ground))
# print(ground_truth_output)
# print(pred_ground)
pred_fault = results.predict(fault_input)
print(mse(fault_output,pred_fault))

0.07160070904597662
1.8469863013976193


# Experiment: Feedback loop for data collection

In [503]:
#[1.189449	0.223208	5.089774	1.797438	0.180184]
i = 97
data2 = data[95:97]
data2 = data2.reset_index(drop=True)
stats = data2.describe()
while stats['outdoor_temp']['std'] < 1 or stats['zone_temp']['std'] < 0.2 or stats['occupancy']['std'] < 3  or stats['supply_temp']['std'] < 1 or stats['air_flow']['std'] < 0.1:
    if i > len(data):
        break
    i = i + 1
    data2 = data[95:i]
    data2 = data2.reset_index(drop=True)
    stats = data2.describe()
# data2['outdoor_temp'] = data2['outdoor_temp'] +273
# data2['zone_temp'] = data2['zone_temp'] +273
# data2['supply_temp'] = data2['supply_temp'] +273
data2.describe()

,outdoor_temp,zone_temp,occupancy,supply_temp,air_flow
count,12.000000,12.000000,12.000000,12.000000,12.000000
mean,272.351389,294.280722,8.333333,295.090751,0.672822
std,1.100768,0.226192,3.892495,1.437726,0.137799
min,270.900000,294.100013,0.000000,294.180418,0.377812
25%,271.454167,294.100105,10.000000,294.318448,0.731824
50%,272.325000,294.167116,10.000000,294.497442,0.731824
75%,273.195833,294.424213,10.000000,294.814783,0.731824
max,274.066667,294.724292,10.000000,298.134578,0.731824


In [504]:
inputs, output = define_vectors(data2)
data2_test = data[i:i+12]
data2_test = data2_test.reset_index(drop=True)
inputs_test, output_test = define_vectors(data2_test)

In [508]:
scaler1 = StandardScaler()
scaler2 = StandardScaler()

inputs = pd.DataFrame(data = inputs)
inputs = sm.add_constant(inputs) #We add constant because statsmodel library requires us. Otherwise, it would take the form: y = mx.
output = pd.DataFrame(data = output)

inputs_test = pd.DataFrame(data = inputs_test)
inputs_test = sm.add_constant(inputs_test,has_constant='add')
output_test = pd.DataFrame(data = output_test)

inputs_scaled = scaler1.fit_transform(inputs)
output_scaled = scaler2.fit_transform(output)

inputs_scaled_test = scaler1.transform(inputs_test)
output_scaled_test = scaler2.transform(output)

In [509]:
model = sm.OLS(output_scaled, inputs_scaled)
results = model.fit()
print(results.summary())
print(results.params)

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.990
Model:                            OLS   Adj. R-squared (uncentered):              0.987
Method:                 Least Squares   F-statistic:                              274.3
Date:                Thu, 06 May 2021   Prob (F-statistic):                    2.11e-08
Time:                        23:25:06   Log-Likelihood:                          9.9282
No. Observations:                  11   AIC:                                     -13.86
Df Residuals:                       8   BIC:                                     -12.66
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

/Users/devanshukumar/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
/Users/devanshukumar/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1860: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


In [510]:
pred = results.predict(inputs_scaled_test)
err = mse(output_test,pred)
print(err)

0.8880577326371111
